In [5]:
import pandas as pd

df = pd.read_csv('shuffled_movie_data.csv')
df.head()
#df.iloc[0]['review']

,review,sentiment
0,"Now, this movie is the worst i have ever seen!...",0
1,"Slow, boring, extremely repetitive. No wonder ...",0
2,"For those fans of Laurel and Hardy, the 1940s ...",0
3,First things first - though I believe Joel Sch...,0
4,"This is one gripping movie, reminiscent of the...",1


In [6]:
import numpy as np

#Aleatorizamos las entradas
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df[['review','sentiment']].to_csv('shuffled_movie_data.csv',index=False)
df.head()

,review,sentiment
11841,I have seen this movie and in all honestly was...,0
19602,I only saw this recently but had been aware of...,1
45519,Albert Pyun delivers a very good action/drama ...,1
25747,This is a truly great and beautiful movie. The...,1
42642,"You know, I'm sure the boys were sitting aroun...",0


Preprocesamiento

In [15]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    #print(text.split())
    text = [w for w in text.split() if w not in stop]
    
    tokenized = [porter.stem(w) for w in text]
    return text
    #return ['good','nice','bad',"don't like"]
#tokenizer(df.iloc[1]['review'])
#print(type(tx))

In [8]:
# Prueba del tokenizer
# tokenizer(df.iloc[5]['review'])

In [9]:
def stream_docs(path):
    with open(path,'r') as csv:
        next(csv)
        for line in csv:
            text,label = line[:-3],int(line[-2])
            yield text, label

#stream = stream_docs('shuffled_movie_data.csv')


In [10]:
#next(stream_docs(path='shuffled_movie_data.csv'))

In [11]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

## EJERCICIO 1

Agregar nuevos Features

Según https://web.stanford.edu/~jurafsky/slp3/7.pdf

In [61]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='shuffled_movie_data.csv')

from nltk.util import ngrams
from nltk import FreqDist

data = ["this", "is", "not", "a", "test", "this", "is", "real", "not", "a", "test", "this", "is", "this", "is", "real", "not", "a", "test"]

from scipy.stats import itemfreq


dict = {}
for size in 1,2,3:
    dict[size] = FreqDist(ngrams(data,size))
    
l =[]

for values in dict.values():
    l += list(values.values())

##PASOS PARA GENERAR EL ALGORITMO
###TOKENIZAR CADA PARRAFO DE ENTRENAMIENTO
#####HACER LA TOKENIZACION CON 1-, 2-, Y 3- GRAMS
###ENCONTRAR LAS FRENCUENCIAS DE DISTRIBUCION PARA CADA TOKEN

### SUMAR LOS TOKENS IGUALES EN TODOS LOS DATOS DE ENTRAMIENTO
### ORDENARLOS Y TRABAJAR CON SOLO LOS N MAYORES

### CREAR UN VECTOR DE PESOS ALEATORIOS ENTRE 0-1 Y EMPEZAR EL 
### EVALUAR

### CREAR EL ALGORITMO STOCHASTIC GRANDIENT ASCENT Y EMPEZAR
### EL PROCESAMIENTO CON BATCHS DE 1000 O MAS
### ITERAR SOBRE LOS 45000 DATOS DE ENTRENAMIENTO

### ETAPA DE PRUEBAS
### PROCESAR LOS DATOS RESTANTES Y EVALUAR LA CANTIDAD DE
### ACIERTOS EXPRESANDO EL RESULTADO EN FORMA DE PORCENTAJE

### PARA EL EJERCICIO 1: TAN SOLO VARIAR LOS TOKENS, Y AGREGAR
### UNOS CUANTOS NUEVOS


##### ENCONTRANDO LAS PALABRAS MAS USADAS EN TODO EL CORPUS
##### USAMOS LA FUNCION TOKENIZER QUE FILTRA STOPWORDS



#tokenizer(df.iloc[0]['review'])

n = 50000 #numero de ejemplos muestrados
print("Tokenizing Opinions")
corpus_words = [] ## Examples tokenized
for i in range(n):
    corpus_words += [tokenizer(df.iloc[i]['review'])]
    if i %10000 == 0:
        print("Completed ",20*i/10000,"%")
#print(corpus_words)

word_dict = {}
for size in 1,2,3,4,5:
    word_dict[size] =  FreqDist(ngrams(corpus_words[0],size))

print("Adding up all values")
for i in range(1,n):
    for size in 1,2,3,4,5:
        word_dict[size].update(FreqDist(ngrams(corpus_words[i],size)))
    if i %10000 == 0:
        print("Completed ",20*i/10000,"%")

#for i in 1,2,3,4,5:
#    print(word_dict[i].most_common(30))
    
#Supongamos que nuestras características son los 10 ngramas (n[1,5])
#mas frecuentes en toda la base de datos






Tokenizing Opinions
Completed  0.0 %
Completed  20.0 %
Completed  40.0 %
Completed  60.0 %
Completed  80.0 %
Adding up all values
Completed  20.0 %
Completed  40.0 %
Completed  60.0 %
Completed  80.0 %


In [94]:
#word_dict[3].most_common(2)[0][0]

## Aca debemos contar la ocurrencia de los n gramas obtenidos y guardarlos
## para formar un vector para cada dato de ejemplo

number_features = 0
keys = []
for i in 1,2,3,4,5:
    number_features+= 90-10*i
    tmp = word_dict[i].most_common(90-10*i)
    for j in range(90-10*i):
        keys += (tmp[j][0],)
#keys
#number_features = 300!
    

300


In [97]:
#print(keys)
def get_vectors(_start,_stop,number_features):
    text_tmp_dict = {}
    for i in range(number_features):
        text_tmp_dict[keys[i]] = i
    #print(text_tmp_dict)    
    
    trainingSet_length = _stop - _start
    trainingSet = []
    for example in range(_start,_stop):
        text_tmp = corpus_words[example]
        #print(text_tmp)
        #print("Occurency in Example")

        word_vector = [0 for i in range(number_features)]
        #print(word_vector)

        for i in 1,2,3,4,5:
            for token in ngrams(text_tmp,i):
                if token in keys:
                    #print(token)
                    word_vector[text_tmp_dict[token]] += 1
                    #text_tmp_dict[token][1] += 1
        #print(word_vector)
        trainingSet += [word_vector]
    #El training set que se retorna contiene una columna mas debido al BIAS
    trainingSet = np.array(trainingSet)
    trainingSet = np.c_[np.ones(trainingSet_length),trainingSet]
    return trainingSet

In [206]:
import math
##DEFINIMOS NUESTRA FUNCIÓN LOGISTICA(SIGMOIDE)
def sigmoid(gamma):
    if gamma < 0:
        return 1 - 1 / (1 + math.exp(gamma))
    return 1 / (1 + math.exp(-gamma))

sigmoid = np.vectorize(sigmoid,otypes=[np.float])

## Obteniendo los valores de los test
b= np.array([df['sentiment']])

b = b.T ## REPRESENTA A LOS VALORES TARGET
print(b)


## Inicializamos los pesos en forma aleatoria
import random


weights = np.full((number_features+1, 1), 0.0)
for i in range( number_features+1 ):
    weights[i] = round(random.uniform(-1,1),2)

#sigmoid(np.dot(get_vectors(0,50,number_features),weights))

# Haciendo la regresion para la función logística



[[0]
 [1]
 [1]
 ..., 
 [0]
 [0]
 [1]]


In [207]:
## Implementamos el estocastic gradient ascent
## Evaluamos hasta los primeros 45000 ejemplos
print("Empezando el Entrenamiento")
alpha = 0.0001
for batch in range(45):
    start = 1000*batch
    stop = 1000*(batch+1)
    #print(start,stop)
    vectors = get_vectors(start,stop,number_features)
    #print(vectors.shape)
    tmp_result = sigmoid(np.dot(vectors,weights))
    #print(tmp_result)
    for i in range(start,stop):
        for w in range(number_features + 1):
            weights[w] = weights[w] + alpha * (b[i][0]-tmp_result[i%1000][0]) * vectors[i%1000][w]
        #print(weights[0:5])
    print("Completado al",round(100/ 45,2) *batch, "%")
    

Empezando el Entrenamiento
Completado al 0.0 %
Completado al 2.22 %
Completado al 4.44 %
Completado al 6.66 %
Completado al 8.88 %
Completado al 11.100000000000001 %
Completado al 13.32 %
Completado al 15.540000000000001 %
Completado al 17.76 %


KeyboardInterrupt: 

In [193]:
## testeamos el accuracy

test_result = sigmoid(np.dot(get_vectors(45000,50000,number_features),weights))
print(test_result[:5],b[45000:45000+5])

acc = 0.0
for i in range(0,5000):
    acc += abs(test_result[i][0] - b[45000+i][0])
print(acc/5000)



[[ 0.73689584]
 [ 0.70176302]
 [ 0.00107816]
 [ 0.31747291]
 [ 0.12355011]] [[1]
 [1]
 [0]
 [0]
 [1]]
0.529055795133
